In [1]:
#Design for the board:
# 0 is an empty cell that can be traversed onto
# 1 is an obstacle
# 2 is the starting point
# 3 is the end point


board = [
    [2,0,0,0,1],
    [0,0,0,1,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [1,0,0,0,3]
]

start = board[0][0]
end = board[4][4]

#algorithms will work for any other rectangular/square board
def getStartPos(board):
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == 2:
                return (i,j)
            
def getEndPos(board):
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == 3:
                return (i,j)

In [2]:
#bfs and dfs search
class Node: 
    
    #row, col, path
    def __init__(self, i, j, path):
        self.pos = (i,j)
        self.path = path # a list of coordinates of nodes visited on this route

    #private method, only to be called by getNextNodes
    def makeNodesFromPos(self, lop, newPath):
        nextNodesList = []
        for k in range(len(lop)):
            coord = lop[k]
            nextNodesList.append(Node(coord[0], coord[1], newPath))
        return nextNodesList

    def getNextNodes(self, board):
        currentNode = self
        
        m = len(board) - 1
        n = len(board[0]) - 1
        temp = []
        i,j = currentNode.pos
        
        possibilities = [(i+1,j), (i, j+1), (i-1,j), (i, j-1)] #currently not allowing for diagonal movement
        for coord in possibilities:
                if coord not in currentNode.path:
                    if 0 <= coord[0] <= m:
                        if 0 <= coord[1] <= n:
                            if board[coord[0]][coord[1]] == 0 or board[coord[0]][coord[1]] == 3:
                                temp.append(coord)

        newPath = currentNode.path.copy()
        newPath.append(currentNode.pos)

        
        nextNodesList = currentNode.makeNodesFromPos(temp, newPath)
        

        return nextNodesList
        
        

###################################################################################################

#bfs search
def bfsSolve(board, startPos):
    (a,b) = startPos
    startNode = Node(a, b, [])
    queue = [startNode]
    while len(queue) > 0:
        currentNode = queue[0]
        (x,y) = currentNode.pos
        if board[x][y] == 3: #if this node is the end
            return currentNode.path
        else:
            queue = queue[1:]
            queue = queue + currentNode.getNextNodes(board)
            
    print("no solution")
    return False

#The only difference between dfs and bfs is the order in which pending items are added to the queue

#dfs solver
def dfsSolve(board, startPos):
    (a,b) = startPos
    startNode = Node(a, b, []) 
    queue = [startNode]
    while len(queue) > 0:
        currentNode = queue[0]
        (x,y) = currentNode.pos
        if board[x][y] == 3: #if this node is the end
            return currentNode.path
        else:
            queue = queue[1:]
            queue = currentNode.getNextNodes(board) + queue #This is the difference between dfs and bfs
            
    print("no solution")
    return False

In [3]:
board = [
    [2,0,0,0,1],
    [0,0,0,1,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [1,0,0,0,3]
]

start = getStartPos(board)
end = getEndPos(board)
bfsSolve(board, start)

[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (4, 2), (4, 3)]

In [4]:
board = [
    [2,0,0,0,1],
    [0,0,0,1,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [1,0,0,0,3]
]

start = getStartPos(board)
end = getEndPos(board)
dfsSolve(board, start)

[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (4, 2), (4, 3)]

In [5]:
from queue import PriorityQueue
import sys

class DNode:
    # Node val(type), row, col, node weight, previous node
    def __init__(self, val, i, j, weight, prev):
        self.val = val
        self.pos = (i,j)
        self.weight = weight
        self.prev = prev
        self.neighbors = []
        self.visited = False
        
    def __lt__(self, other): 
        return self.weight < other.weight #this method is for PriorityQueue to arrange nodes properly
        
    #REQUIRES: the board has already been converted into a board of nodes
    def addNeighborNodes(self, board):
        m = len(board) - 1
        n = len(board[0]) - 1
        (i,j) = self.pos
        possibilities = [(i+1,j), (i, j+1), (i-1,j), (i, j-1)] #currently not allowing for diagonal neighbors either
        for coord in possibilities:
            if 0 <= coord[0] <= m:
                if 0 <= coord[1] <= n:
                    if board[coord[0]][coord[1]].val == 0 or board[coord[0]][coord[1]].val == 3:
                        self.neighbors.append(board[coord[0]][coord[1]])
                    

###################################################################################################
# Sig: 2Dlist, tuple, tuple
def DijkstraSolve(board, startPos, endPos):
    maxInt = sys.maxsize #this is basicalle Integer.maxInt
    queue = PriorityQueue()
    (startRow,startCol) = startPos
    (endRow,endCol) = endPos
    
    
    #turning the board into a graph
    for i in range(len(board)):
        for j in range(len(board[0])):
            val = board[i][j]
            board[i][j] = DNode(val,i,j,maxInt,None)
            
    for i in range(len(board)):
        for j in range(len(board[0])):
            board[i][j].addNeighborNodes(board)
            

    board[startRow][startCol].weight = 0 #starting weight = 0
    
    board[endRow][endCol].neighbors = [] #ending node shouldn't point anywhere else
    
    
    #adding all the nodes into the priority queue
    for i in range(len(board)):
        for j in range(len(board[0])):
            queue.put((board[i][j].weight, board[i][j]))
    
    #the actual algorithm itself
    while not queue.empty(): 
        (weight, currentNode) = queue.get()
        currentNode.visited = True
        
        neighbors = currentNode.neighbors
        for neighbor in neighbors:
            if neighbor.weight > currentNode.weight + 1:
                neighbor.weight = currentNode.weight + 1
                neighbor.prev = currentNode.pos
                
        
        #re-order the queue by removing all the elements and putting it back in
        queue = PriorityQueue()
        for i in range(len(board)):
            for j in range(len(board[0])):
                if not board[i][j].visited:
                    queue.put((board[i][j].weight, board[i][j]))
        
        
    if board[endRow][endCol].weight == maxInt: #the end node was never reached
        print("no solution")
        return False
    
    path = []
    (curRow, curCol) = (endRow, endCol) #just for readability
    
    while board[curRow][curCol].prev != None: 
        path.append(board[curRow][curCol].pos)
        (curRow,curCol) = board[curRow][curCol].prev
    return path

In [6]:
board = [
    [2,0,0,0,1],
    [0,0,0,1,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [1,0,0,0,3]
]

start = getStartPos(board)
end = getEndPos(board)
DijkstraSolve(board, start, end)

[(4, 4), (3, 4), (3, 3), (3, 2), (3, 1), (2, 1), (1, 1), (0, 1)]

In [7]:
from queue import PriorityQueue
import sys

class ANode(DNode): #class ANode extends DNode
    def __init__(self, val, i, j, weight, prev, heuristic):
        super().__init__(val, i, j, weight, prev)
        self.heuristic = heuristic
        
    def __lt__(self, other):
        if self.weight == sys.maxsize:
            return False
        elif other.weight == sys.maxsize:
            return True
        else:
            return (self.weight + self.heuristic) < (other.weight + other.heuristic)

###################################################################################################

# Sig: 2Dlist, tuple, tuple
def AStarSolve(board, startPos, endPos):
    maxInt = sys.maxsize #this is basicalle Integer.maxInt
    queue = PriorityQueue()
    (startRow,startCol) = startPos
    (endRow,endCol) = endPos
    
    
    #turning the board into a graph
    for i in range(len(board)):
        for j in range(len(board[0])):
            val = board[i][j]
            heuristic = abs(endRow - i) + abs(endCol - j) #minimum number of moves needed to reach desitnation
            board[i][j] = ANode(val,i,j,maxInt,None, heuristic)
            
    for i in range(len(board)):
        for j in range(len(board[0])):
            board[i][j].addNeighborNodes(board)
            

    board[startRow][startCol].weight = 0 #starting weight = 0
    board[endRow][endCol].neighbors = [] #ending node shouldn't point anywhere else
    
    
    #adding all the nodes into the priority queue
    for i in range(len(board)):
        for j in range(len(board[0])):
            queue.put((board[i][j].weight, board[i][j]))
    
    #the actual algorithm itself
    while not queue.empty(): 
        (weight, currentNode) = queue.get()
        currentNode.visited = True
        
        if currentNode.val == 3: #this is the end position
            path = []
            (curRow, curCol) = (endRow, endCol) 

            while board[curRow][curCol].prev != None: 
                path.append(board[curRow][curCol].pos)
                (curRow,curCol) = board[curRow][curCol].prev
            return path
        
        neighbors = currentNode.neighbors
        for neighbor in neighbors:
            if neighbor.weight > currentNode.weight + 1:
                neighbor.weight = currentNode.weight + 1
                neighbor.prev = currentNode.pos
                
        
        #re-order the queue by removing all the elements and putting it back in...
        queue = PriorityQueue()
        for i in range(len(board)):
            for j in range(len(board[0])):
                if not board[i][j].visited:
                    queue.put((board[i][j].weight, board[i][j]))
        
        
    print("no solution")
    return False

In [8]:
board = [
    [2,0,0,0,1],
    [0,0,0,1,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [1,0,0,0,3]
]

start = getStartPos(board)
end = getEndPos(board)
AStarSolve(board,start,end)

[(4, 4), (3, 4), (3, 3), (3, 2), (3, 1), (2, 1), (1, 1), (0, 1)]